<a href="https://colab.research.google.com/github/jojosam90/auto-encoder/blob/main/auto-encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TensorBoard

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Loading Fashion MNIST train and test dataset, normalize and reshape 

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train,_) ,(x_test,_) = fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train = np.reshape(x_train,(len(x_train),28,28,1))
x_test = np.reshape(x_test,(len(x_test),28,28,1))

## Buidling encoder 
### Encoder encodes high dimensional input into lower-dimensional latent (bottleneck)

### INPUT

In [4]:
# pass gray scale input image of size (28,28,1)
inputs = tf.keras.Input(shape=(28,28,1),name ='input_layer')

### ENCODER

In [5]:
# Conv Block 1 -> BatchNorm -> leaky Relu
encoded = tf.keras.layers.Conv2D(32,kernel_size =3,strides =1,padding='same',name='conv_1')(inputs)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_1')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_1')(encoded)

# Conv Block 2 -> BatchNorm -> leaky Relu
encoded = tf.keras.layers.Conv2D(64,kernel_size =3,strides =2,padding='same',name='conv_2')(encoded)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_2')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_2')(encoded)

# Conv Block 3 -> BatchNorm -> leaky Relu
encoded = tf.keras.layers.Conv2D(64,kernel_size =3,strides =2,padding='same',name='conv_3')(encoded)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_3')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_3')(encoded)


## Buidling decoder 
### Decoder will decompress latent representation to recreate input data.

### DECODER

In [6]:
# DeConv Block 1 -> BatchNorm -> leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(64,3,strides =1,padding='same',name='conv_transpose_1')(encoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_4')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_4')(decoded)

# DeConv Block 2 -> BatchNorm -> leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(64,3,strides =2,padding='same',name='conv_transpose_2')(decoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_5')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_5')(decoded)

# DeConv Block 3 -> BatchNorm -> leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(32,3,2,padding='same',name='conv_transpose_3')(decoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_6')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_6')(decoded)

### OUTPUT

In [7]:
# output (use sigmoid activation functions as it flattens the output to be in range [0,1])
outputs = tf.keras.layers.Conv2DTranspose(1,3,1,padding='same', activation ='sigmoid', name ='conv_transpose_4')(decoded)

In [8]:
# Define Structural Similarity Index (SSIM) Loss Function
def SSIMLoss(y_true,y_pred):
  return 1-tf.reduce_mean(tf.image.ssim(y_true,y_pred,1.0))

In [9]:
# Define Autoencoder : Optimiser: Adam , Loss: SSIM Loss
autoencoder = tf.keras.Model(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(lr = 0.0005)
autoencoder.compile(optimizer = optimizer, loss = SSIMLoss)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [13]:
# Setting up TENSORBOARD as callback, for logging loss metric and Training
!rm -rf ./tmp/ # clear previous log
from keras.callbacks import TensorBoard
hsit = autoencoder.fit(x_train, x_train,
                       epochs = 10,
                       batch_size = 128,
                       shuffle= True,
                       validation_data = (x_test,x_test), callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Epoch 1/10
469/469 [==============================] - 31s 38ms/step - loss: 0.0851 - val_loss: 0.2106
Epoch 2/10
469/469 [==============================] - 17s 36ms/step - loss: 0.0278 - val_loss: 0.0233
Epoch 3/10
469/469 [==============================] - 17s 36ms/step - loss: 0.0198 - val_loss: 0.0182
Epoch 4/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0162 - val_loss: 0.0159
Epoch 5/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0138 - val_loss: 0.0117
Epoch 6/10
469/469 [==============================] - 17s 36ms/step - loss: 0.0122 - val_loss: 0.0119
Epoch 7/10
469/469 [==============================] - 17s 36ms/step - loss: 0.0112 - val_loss: 0.0139
Epoch 8/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0102 - val_loss: 0.0125
Epoch 9/10
469/469 [==============================] - 17s 36ms/step - loss: 0.0095 - val_loss: 0.0112
Epoch 10/10
469/469 [==============================] - 17s 37ms/step - loss: 0.008